# Dissertation Data Querying
### Gavin Rolls

This script contains the queries I used to save OSM and Overture data locally for Greater London and Birmingham. Because it's easier to grab the data I need from my desktop as opposed to running these queries every time I need data, I'm leaving these queries here but only intend to use them once

## Library Imports & Database Configuration

In [14]:
#Library Imports - using DuckDB for Overture Import

#Database
import duckdb

#Basics
import pandas as pd
import geopandas as gpd
from geopy.geocoders import Nominatim

#OpenStreetMap
import osmnx as ox

In [15]:
#Config SQL
%pip install ipython-sql duckdb duckdb-engine jupysql --quiet
%pip install --upgrade grpcio --quiet
%load_ext sql

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
There's a new jupysql version available (0.10.11), you're running 0.10.10. To upgrade: pip install jupysql --upgrade
Deploy Shiny apps for free on Ploomber Cloud! Learn more: https://ploomber.io/s/signup


In [16]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///:memory:

In [17]:
%%sql      
INSTALL httpfs;

LOAD httpfs;

INSTALL spatial;

LOAD spatial;

SET s3_region='us-west-2';

,Success


## Get Bounding Boxes

In [25]:
#Get London Bounding Box

# Initialize the geolocator
geolocator = Nominatim(user_agent="geoapi")

# Get location data for London
location = geolocator.geocode("London")

# Get the bounding box
bounding_box_london = location.raw['boundingbox']

min_lat_london, max_lat_london = float(bounding_box_london[0]), float(bounding_box_london[1])
min_lon_london, max_lon_london = float(bounding_box_london[2]), float(bounding_box_london[3])

print("London Bounding Box Coordinates:")
print(f"Min Longitude: {min_lon_london}")
print(f"Min Latitude: {min_lat_london}")
print(f"Max Longitude: {max_lon_london}")
print(f"Max Latitude: {max_lat_london}")
print()

# Get location data for Birmingham
location_birmingham = geolocator.geocode("Birmingham")

# Get the bounding box
bounding_box_birmingham = location_birmingham.raw['boundingbox']

# Convert bounding box to coordinates for Birmingham
min_lat_birmingham, max_lat_birmingham = float(bounding_box_birmingham[0]), float(bounding_box_birmingham[1])
min_lon_birmingham, max_lon_birmingham = float(bounding_box_birmingham[2]), float(bounding_box_birmingham[3])

print("Birmingham Bounding Box Coordinates:")
print(f"Min Longitude: {min_lon_birmingham}")
print(f"Min Latitude: {min_lat_birmingham}")
print(f"Max Longitude: {max_lon_birmingham}")
print(f"Max Latitude: {max_lat_birmingham}")


London Bounding Box Coordinates:
Min Longitude: -0.5103751
Min Latitude: 51.2867601
Max Longitude: 0.3340155
Max Latitude: 51.6918741

Birmingham Bounding Box Coordinates:
Min Longitude: -2.0336486
Min Latitude: 52.381053
Max Longitude: -1.7288417
Max Latitude: 52.6087058


## Download POIs - Overture

### London

In [36]:
%%sql

COPY (
    SELECT
        names.primary AS name,
        categories.main as category,
        ROUND(confidence,2) as confidence,
        ST_GeomFromWKB(geometry) as geometry
FROM read_parquet('s3://overturemaps-us-west-2/release/2024-05-16-beta.0/theme=places/*/*')
WHERE
    bbox.xmin BETWEEN -0.5103751 AND 0.3340155 AND
    bbox.ymin BETWEEN 51.2867601 AND 51.6918741
) TO 'data/overture_data/london_places.geojson' WITH (FORMAT GDAL, DRIVER 'GeoJSON', SRS 'EPSG:4326');

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


### Birmingham

In [37]:
%%sql

COPY (
    SELECT
        names.primary AS name,
        categories.main as category,
        ROUND(confidence,2) as confidence,
        ST_GeomFromWKB(geometry) as geometry
FROM read_parquet('s3://overturemaps-us-west-2/release/2024-05-16-beta.0/theme=places/*/*')
WHERE
    bbox.xmin BETWEEN -2.0336486 AND -1.7288417 AND
    bbox.ymin BETWEEN 52.381053 AND 52.6087058
) TO 'data/overture_data/bham_places.geojson' WITH (FORMAT GDAL, DRIVER 'GeoJSON', SRS 'EPSG:4326');

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


### Overture Download Stats

In [51]:
#Basic overview stats of London Data - Using the locally saved files here
london_places = gpd.read_file('data/overture_data/london_places.geojson')
bham_places = gpd.read_file('data/overture_data/bham_places.geojson')

#Count of Features
print("London POI Count: " + str(london_places.shape[0]))
print("Birmingham POI Count: " + str(bham_places.shape[0]))

London POI Count: 343712
Birmingham POI Count: 46781


## Download Buildings - OSM

### London

In [83]:
#Download Data for 32 Boroughs (Exc. City of London)
place_name = 'London, United Kingdom'

#Test new config
ox.config(use_cache=True, log_console=True)

buildings = ox.features_from_place(place_name, tags={'building': True})
buildings = buildings[buildings.geometry.notnull()]
building_footprints = buildings[buildings.geom_type.isin(['Polygon', 'MultiPolygon'])]

for col in building_footprints.columns:
    if building_footprints.apply(lambda x: isinstance(x, list)).any():
        building_footprints = building_footprints[col].apply(lambda x: str(x) if isinstance(x, list) else x)

building_footprints = building_footprints[['name', 'geometry']].reset_index()
print(building_footprints)
        
# Save the combined GeoDataFrame to a geojson file
building_footprints.to_file("data/osm_data/boroughs_buildings.geojson", driver="GeoJSON")

/var/folders/2b/877nyyk109x4dd8hgx4ckwl40000gn/T/ipykernel_13639/2344882920.py:5: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.config(use_cache=True, log_console=True)


ReadTimeout: HTTPSConnectionPool(host='overpass-api.de', port=443): Read timed out. (read timeout=180)

### City of London

In [89]:
#Add City of London
place_name = 'City of London, England, United Kingdom'

buildings_city = ox.features_from_place(place_name, tags={'building': True})
buildings_city = buildings_city[buildings_city.geometry.notnull()]
building_footprints_city = buildings_city[buildings_city.geom_type.isin(['Polygon', 'MultiPolygon'])]

for col in building_footprints_city.columns:
    if building_footprints_city.apply(lambda x: isinstance(x, list)).any():
        building_footprints_city = building_footprints_city[col].apply(lambda x: str(x) if isinstance(x, list) else x)

building_footprints_city = building_footprints_city[['name', 'geometry']].reset_index()

building_footprints_combined = pd.concat([building_footprints, building_footprints_city], ignore_index=True)
        
# Save the combined GeoDataFrame to a geojson file
building_footprints_combined.to_file("data/osm_data/all_london_buildings.geojson", driver="GeoJSON")

### Birmingham

In [90]:
#Download Data for Birmingham
place_name = 'Birmingham, United Kingdom'

#Test new config
ox.config(use_cache=True, log_console=True)

buildings = ox.features_from_place(place_name, tags={'building': True})
buildings = buildings[buildings.geometry.notnull()]
building_footprints = buildings[buildings.geom_type.isin(['Polygon', 'MultiPolygon'])]

for col in building_footprints.columns:
    if building_footprints.apply(lambda x: isinstance(x, list)).any():
        building_footprints = building_footprints[col].apply(lambda x: str(x) if isinstance(x, list) else x)

building_footprints = building_footprints[['name', 'geometry']].reset_index()
print(building_footprints)
        
# Save the combined GeoDataFrame to a geojson file
building_footprints.to_file("data/osm_data/bham_buildings.geojson", driver="GeoJSON")

/var/folders/2b/877nyyk109x4dd8hgx4ckwl40000gn/T/ipykernel_13639/2788206819.py:5: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.config(use_cache=True, log_console=True)


       element_type     osmid                                          name  \
0               way  16966777                                           NaN   
1               way  16966784                                           NaN   
2               way  16966785                                           NaN   
3               way  16966788                                           NaN   
4               way  16966789                                           NaN   
...             ...       ...                                           ...   
256274     relation  15809922                                       Block A   
256275     relation  15823526                                           NaN   
256276     relation  16414329  Birmingham Settlement - 610 Community Centre   
256277     relation  16542773                                           NaN   
256278     relation  17785650                             2 Centenary Drive   

                                                 ge

### Extract Building Type Info from OSM - Commercial


In [91]:
#Download Data for 32 Boroughs (Exc. City of London)
place_name = 'London, United Kingdom'

commercial_buildings = ox.features_from_place(place_name, tags={'building': ['commercial']})

commercial_buildings = commercial_buildings[commercial_buildings.geometry.notnull()]
commercial_building_footprints = commercial_buildings[commercial_buildings.geom_type.isin(['Polygon', 'MultiPolygon'])]

for col in commercial_building_footprints.columns:
    if commercial_building_footprints.apply(lambda x: isinstance(x, list)).any():
        commercial_building_footprints = commercial_building_footprints[col].apply(lambda x: str(x) if isinstance(x, list) else x)

commercial_building_footprints = commercial_building_footprints[['name', 'geometry']].reset_index()
print(commercial_building_footprints)
        
# Save the combined GeoDataFrame to a geojson file
commercial_building_footprints.to_file("data/osm_data/boroughs_commercial_buildings.geojson", driver="GeoJSON")

     element_type     osmid                      name  \
0             way   4076571  Bromley Road Retail Park   
1             way   4076587   Catford Delivery Office   
2             way   5790535              Refuge House   
3             way   5986754         One Canada Square   
4             way   5987126                       NaN   
...           ...       ...                       ...   
5294     relation  13472609                Hyde House   
5295     relation  13535999           Old Guy's House   
5296     relation  15685271                       NaN   
5297     relation  15761586               China Works   
5298     relation  16030676           Camelford House   

                                               geometry  
0     POLYGON ((-0.02063 51.43669, -0.02047 51.43631...  
1     POLYGON ((-0.02001 51.43859, -0.02004 51.43865...  
2     POLYGON ((-0.07800 51.65301, -0.07777 51.65303...  
3     POLYGON ((-0.02002 51.50477, -0.01996 51.50500...  
4     POLYGON ((-0.02588 

In [92]:
#Add City of London
place_name = 'City of London, England, United Kingdom'

commercial_buildings_city = ox.features_from_place(place_name, tags={'building': ['commercial']})
commercial_buildings_city = commercial_buildings_city[commercial_buildings_city.geometry.notnull()]
commercial_building_footprints_city = commercial_buildings_city[commercial_buildings_city.geom_type.isin(['Polygon', 'MultiPolygon'])]

for col in commercial_building_footprints_city.columns:
    if commercial_building_footprints_city.apply(lambda x: isinstance(x, list)).any():
        commercial_building_footprints_city = commercial_building_footprints_city[col].apply(lambda x: str(x) if isinstance(x, list) else x)

commercial_building_footprints_city = commercial_building_footprints_city[['name', 'geometry']].reset_index()

commercial_building_footprints_combined = pd.concat([commercial_building_footprints, commercial_building_footprints_city], ignore_index=True)
        
# Save the combined GeoDataFrame to a geojson file
commercial_building_footprints_combined.to_file("data/osm_data/all_london_commercial_buildings.geojson", driver="GeoJSON")

In [93]:
#Download Data for Birmingham
place_name = 'Birmingham, United Kingdom'

commercial_buildings = ox.features_from_place(place_name, tags={'building': ['commercial']})

commercial_buildings = commercial_buildings[commercial_buildings.geometry.notnull()]
commercial_building_footprints = commercial_buildings[commercial_buildings.geom_type.isin(['Polygon', 'MultiPolygon'])]

for col in commercial_building_footprints.columns:
    if commercial_building_footprints.apply(lambda x: isinstance(x, list)).any():
        commercial_building_footprints = commercial_building_footprints[col].apply(lambda x: str(x) if isinstance(x, list) else x)

commercial_building_footprints = commercial_building_footprints[['name', 'geometry']].reset_index()
print(commercial_building_footprints)
        
# Save the combined GeoDataFrame to a geojson file
commercial_building_footprints.to_file("data/osm_data/bham_commercial_buildings.geojson", driver="GeoJSON")

    element_type       osmid                     name  \
0            way    17405658                      NaN   
1            way    20002347  Dental Care Partnership   
2            way    22763584            Station House   
3            way    22763760                      NaN   
4            way    22763952                      NaN   
..           ...         ...                      ...   
312          way  1259115143                      NaN   
313          way  1259115144                      NaN   
314          way  1259115145                      NaN   
315     relation      333915      The Custard Factory   
316     relation     7076030                Eagle Two   

                                              geometry  
0    POLYGON ((-1.82146 52.56186, -1.82179 52.56209...  
1    POLYGON ((-1.82680 52.55679, -1.82680 52.55680...  
2    POLYGON ((-1.82027 52.56533, -1.82036 52.56528...  
3    POLYGON ((-1.81624 52.56247, -1.81633 52.56237...  
4    POLYGON ((-1.82061 52.565

### Extract Building Type Info from OSM - Office

In [94]:
# Download Data for 32 Boroughs (Exc. City of London)
place_name = 'London, United Kingdom'

ox.config(use_cache=True, log_console=True)

office_buildings = ox.features_from_place(place_name, tags={'building': ['office']})

office_buildings = office_buildings[office_buildings.geometry.notnull()]
office_building_footprints = office_buildings[office_buildings.geom_type.isin(['Polygon', 'MultiPolygon'])]

for col in office_building_footprints.columns:
    if office_building_footprints.apply(lambda x: isinstance(x, list)).any():
        office_building_footprints[col] = office_building_footprints[col].apply(lambda x: str(x) if isinstance(x, list) else x)

office_building_footprints = office_building_footprints[['name', 'geometry']].reset_index()
print(office_building_footprints)
        
# Save the combined GeoDataFrame to a geojson file
office_building_footprints.to_file("data/osm_data/boroughs_office_buildings.geojson", driver="GeoJSON")


/var/folders/2b/877nyyk109x4dd8hgx4ckwl40000gn/T/ipykernel_13639/4117539463.py:4: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.config(use_cache=True, log_console=True)


     element_type     osmid                       name  \
0             way   4237383                        ITN   
1             way   5251705                        NaN   
2             way   5986805                    HSBC UK   
3             way   5986920                       KPMG   
4             way   5986929                   Barclays   
...           ...       ...                        ...   
1724     relation  12566775                        NaN   
1725     relation  12581563                BBC Studios   
1726     relation  14168517  The Featherstone Building   
1727     relation  15690226                 The Hudson   
1728     relation  16030675             Tintagel House   

                                               geometry  
0     POLYGON ((-0.11445 51.52304, -0.11478 51.52344...  
1     POLYGON ((-0.23366 51.40761, -0.23281 51.40822...  
2     POLYGON ((-0.01792 51.50522, -0.01794 51.50525...  
3     POLYGON ((-0.01680 51.50508, -0.01663 51.50569...  
4     POLYGON

In [95]:
#Add City of London
place_name = 'City of London, England, United Kingdom'

office_buildings_city = ox.features_from_place(place_name, tags={'building': ['office']})
office_buildings_city = office_buildings_city[office_buildings_city.geometry.notnull()]
office_building_footprints_city = office_buildings_city[office_buildings_city.geom_type.isin(['Polygon', 'MultiPolygon'])]

for col in office_building_footprints_city.columns:
    if office_building_footprints_city.apply(lambda x: isinstance(x, list)).any():
        office_building_footprints_city = office_building_footprints_city[col].apply(lambda x: str(x) if isinstance(x, list) else x)

office_building_footprints_city = office_building_footprints_city[['name', 'geometry']].reset_index()

office_building_footprints_combined = pd.concat([office_building_footprints, office_building_footprints_city], ignore_index=True)
        
# Save the combined GeoDataFrame to a geojson file
office_building_footprints_combined.to_file("data/osm_data/all_london_office_buildings.geojson", driver="GeoJSON")

In [96]:
#Download Data for Birmingham
place_name = 'Birmingham, United Kingdom'

ox.config(use_cache=True, log_console=True)

office_buildings = ox.features_from_place(place_name, tags={'building': ['office']})

office_buildings = office_buildings[office_buildings.geometry.notnull()]
office_building_footprints = office_buildings[office_buildings.geom_type.isin(['Polygon', 'MultiPolygon'])]

for col in office_building_footprints.columns:
    if office_building_footprints.apply(lambda x: isinstance(x, list)).any():
        office_building_footprints[col] = office_building_footprints[col].apply(lambda x: str(x) if isinstance(x, list) else x)

office_building_footprints = office_building_footprints[['name', 'geometry']].reset_index()
print(office_building_footprints)
        
# Save the combined GeoDataFrame to a geojson file
office_building_footprints.to_file("data/osm_data/bham_office_buildings.geojson", driver="GeoJSON")


/var/folders/2b/877nyyk109x4dd8hgx4ckwl40000gn/T/ipykernel_13639/2941556620.py:4: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.config(use_cache=True, log_console=True)


    element_type     osmid                     name  \
0            way  22820818        Baskerville House   
1            way  28292471     No. 1 Colmore Square   
2            way  28425056         McLaren Building   
3            way  28455010        Gazette Buildings   
4            way  28514237                      NaN   
..           ...       ...                      ...   
190     relation   1298933           Fountain Court   
191     relation   3417218     Eight Brindley Place   
192     relation   3417219     Seven Brindley Place   
193     relation   3417220       Ten Brindley Place   
194     relation   4631334  Birmingham Royal Ballet   

                                              geometry  
0    POLYGON ((-1.90809 52.48018, -1.90735 52.48039...  
1    POLYGON ((-1.89668 52.48235, -1.89675 52.48258...  
2    POLYGON ((-1.89265 52.48192, -1.89247 52.48183...  
3    POLYGON ((-1.89313 52.48261, -1.89339 52.48236...  
4    POLYGON ((-1.90172 52.47568, -1.90125 52.47528...

### Extract Building Type Info from OSM - Residential

In [97]:
# Download Data for 32 Boroughs (Exc. City of London)
place_name = 'London, United Kingdom'

ox.config(use_cache=True, log_console=True)

residential_buildings = ox.features_from_place(place_name, tags={'building': ['residential']})

residential_buildings = residential_buildings[residential_buildings.geometry.notnull()]
residential_building_footprints = residential_buildings[residential_buildings.geom_type.isin(['Polygon', 'MultiPolygon'])]

for col in residential_building_footprints.columns:
    if residential_building_footprints.apply(lambda x: isinstance(x, list)).any():
        residential_building_footprints[col] = residential_building_footprints[col].apply(lambda x: str(x) if isinstance(x, list) else x)

residential_building_footprints = residential_building_footprints[['name', 'geometry']].reset_index()
print(residential_building_footprints)
        
# Save the combined GeoDataFrame to a geojson file
residential_building_footprints.to_file("data/osm_data/boroughs_residential_buildings.geojson", driver="GeoJSON")


/var/folders/2b/877nyyk109x4dd8hgx4ckwl40000gn/T/ipykernel_13639/160667686.py:4: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.config(use_cache=True, log_console=True)


      element_type     osmid            name  \
0              way   4948114    Kemble House   
1              way   4948116  Kettleby House   
2              way   4948118   Woolley House   
3              way   5131193             NaN   
4              way   5131194             NaN   
...            ...       ...             ...   
83799     relation  16389021     Birch House   
83800     relation  16389022  Lansdell House   
83801     relation  16847821             NaN   
83802     relation  17193995             NaN   
83803     relation  17729875             NaN   

                                                geometry  
0      POLYGON ((-0.10656 51.46576, -0.10645 51.46582...  
1      POLYGON ((-0.10571 51.46618, -0.10559 51.46624...  
2      POLYGON ((-0.10486 51.46659, -0.10473 51.46665...  
3      POLYGON ((-0.19992 51.46981, -0.19992 51.46987...  
4      POLYGON ((-0.19955 51.46940, -0.19955 51.46980...  
...                                                  ...  
83799  POL

In [98]:
#Add City of London
place_name = 'City of London, England, United Kingdom'

residential_buildings_city = ox.features_from_place(place_name, tags={'building': ['residential']})
residential_buildings_city = residential_buildings_city[residential_buildings_city.geometry.notnull()]
residential_building_footprints_city = residential_buildings_city[residential_buildings_city.geom_type.isin(['Polygon', 'MultiPolygon'])]

for col in residential_building_footprints_city.columns:
    if residential_building_footprints_city.apply(lambda x: isinstance(x, list)).any():
        residential_building_footprints_city = residential_building_footprints_city[col].apply(lambda x: str(x) if isinstance(x, list) else x)

residential_building_footprints_city = residential_building_footprints_city[['name', 'geometry']].reset_index()

residential_building_footprints_combined = pd.concat([residential_building_footprints, residential_building_footprints_city], ignore_index=True)
        
# Save the combined GeoDataFrame to a geojson file
residential_building_footprints_combined.to_file("data/osm_data/all_london_residential_buildings.geojson", driver="GeoJSON")

In [99]:
#Download Data for Birmingham
place_name = 'Birmingham, United Kingdom'

ox.config(use_cache=True, log_console=True)

residential_buildings = ox.features_from_place(place_name, tags={'building': ['residential']})

residential_buildings = residential_buildings[residential_buildings.geometry.notnull()]
residential_building_footprints = residential_buildings[residential_buildings.geom_type.isin(['Polygon', 'MultiPolygon'])]

for col in residential_building_footprints.columns:
    if residential_building_footprints.apply(lambda x: isinstance(x, list)).any():
        residential_building_footprints[col] = residential_building_footprints[col].apply(lambda x: str(x) if isinstance(x, list) else x)

residential_building_footprints = residential_building_footprints[['name', 'geometry']].reset_index()
print(residential_building_footprints)
        
# Save the combined GeoDataFrame to a geojson file
residential_building_footprints.to_file("data/osm_data/bham_residential_buildings.geojson", driver="GeoJSON")


/var/folders/2b/877nyyk109x4dd8hgx4ckwl40000gn/T/ipykernel_13639/2012114761.py:4: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.config(use_cache=True, log_console=True)


       element_type     osmid               name  \
0               way  16966777                NaN   
1               way  16966784                NaN   
2               way  16966785                NaN   
3               way  16966788                NaN   
4               way  16966789                NaN   
...             ...       ...                ...   
189377     relation  11382596      Bentley House   
189378     relation  12197160                NaN   
189379     relation  15809922            Block A   
189380     relation  16542773                NaN   
189381     relation  17785650  2 Centenary Drive   

                                                 geometry  
0       POLYGON ((-1.82461 52.55549, -1.82450 52.55543...  
1       POLYGON ((-1.82324 52.55438, -1.82301 52.55434...  
2       POLYGON ((-1.82273 52.55403, -1.82264 52.55400...  
3       POLYGON ((-1.82341 52.55414, -1.82338 52.55412...  
4       POLYGON ((-1.82311 52.55396, -1.82302 52.55391...  
...            

### Extract Building Type Info from OSM - Retail

In [100]:
# Download Data for 32 Boroughs (Exc. City of London)
place_name = 'London, United Kingdom'

ox.config(use_cache=True, log_console=True)

retail_buildings = ox.features_from_place(place_name, tags={'building': ['retail']})

retail_buildings = retail_buildings[retail_buildings.geometry.notnull()]
retail_building_footprints = retail_buildings[retail_buildings.geom_type.isin(['Polygon', 'MultiPolygon'])]

for col in retail_building_footprints.columns:
    if retail_building_footprints.apply(lambda x: isinstance(x, list)).any():
        retail_building_footprints[col] = retail_building_footprints[col].apply(lambda x: str(x) if isinstance(x, list) else x)

retail_building_footprints = retail_building_footprints[['name', 'geometry']].reset_index()
print(retail_building_footprints)
        
# Save the combined GeoDataFrame to a geojson file
retail_building_footprints.to_file("data/osm_data/boroughs_retail_buildings.geojson", driver="GeoJSON")


/var/folders/2b/877nyyk109x4dd8hgx4ckwl40000gn/T/ipykernel_13639/110674602.py:4: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.config(use_cache=True, log_console=True)


      element_type     osmid                                            name  \
0              way   4270351                                             NaN   
1              way   4425414                                             NaN   
2              way   4425469                                             NaN   
3              way   4770296  Kingston Rowing Club / Tiffin School Boat Club   
4              way   4775619                              St Richard's Court   
...            ...       ...                                             ...   
11690     relation  14208162                   Gulf Ickenham Service Station   
11691     relation  15536955                                             NaN   
11692     relation  16095623                                         Crispin   
11693     relation  17367661                                             NaN   
11694     relation  17745415             Ilford Tesco Superstore Barkingside   

                                       

In [101]:
#Add City of London
place_name = 'City of London, England, United Kingdom'

retail_buildings_city = ox.features_from_place(place_name, tags={'building': ['retail']})
retail_buildings_city = retail_buildings_city[retail_buildings_city.geometry.notnull()]
retail_building_footprints_city = retail_buildings_city[retail_buildings_city.geom_type.isin(['Polygon', 'MultiPolygon'])]

for col in retail_building_footprints_city.columns:
    if retail_building_footprints_city.apply(lambda x: isinstance(x, list)).any():
        retail_building_footprints_city = retail_building_footprints_city[col].apply(lambda x: str(x) if isinstance(x, list) else x)

retail_building_footprints_city = retail_building_footprints_city[['name', 'geometry']].reset_index()

retail_building_footprints_combined = pd.concat([retail_building_footprints, retail_building_footprints_city], ignore_index=True)
        
# Save the combined GeoDataFrame to a geojson file
retail_building_footprints_combined.to_file("data/osm_data/all_london_retail_buildings.geojson", driver="GeoJSON")

In [ ]:
#Download Data for Birmingham
place_name = 'Birmingham, United Kingdom'

ox.config(use_cache=True, log_console=True)

retail_buildings = ox.features_from_place(place_name, tags={'building': ['retail']})

retail_buildings = retail_buildings[retail_buildings.geometry.notnull()]
retail_building_footprints = retail_buildings[retail_buildings.geom_type.isin(['Polygon', 'MultiPolygon'])]

for col in retail_building_footprints.columns:
    if retail_building_footprints.apply(lambda x: isinstance(x, list)).any():
        retail_building_footprints[col] = retail_building_footprints[col].apply(lambda x: str(x) if isinstance(x, list) else x)

retail_building_footprints = retail_building_footprints[['name', 'geometry']].reset_index()
print(retail_building_footprints)
        
# Save the combined GeoDataFrame to a geojson file
retail_building_footprints.to_file("data/osm_data/bham_retail_buildings.geojson", driver="GeoJSON")


/var/folders/2b/877nyyk109x4dd8hgx4ckwl40000gn/T/ipykernel_13639/291704794.py:4: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.config(use_cache=True, log_console=True)
